In [14]:
pip install mysql-connector-python


   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB 330.3 kB/s eta 0:00:50
   ---------------------------------------- 0.1/16.4 MB 787.7 kB/s eta 0:00:21
    --------------------------------------- 0.2/16.4 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.5/16.4 MB 3.0 MB/s eta 0:00:06
   -- ------------------------------------- 0.9/16.4 MB 4.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.9/16.4 MB 4.4 MB/s eta 0:00:04
   ---- ----------------------------------- 2.0/16.4 MB 6.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.3/16.4 MB 6.3 MB/s eta 0:00:03
   ------ --------------------------------- 2.5/16.4 MB 6.6 MB/s eta 0:00:03
   -------- ------------------------------- 3.4/16.4 MB 7.7 MB/s eta 0:00:02
   --------- ------------------------------ 3.9/16.4 MB 8.1 MB/s eta 0:00:02
   ---------- ----------------------------- 4.5/16.4 MB 8.4 MB/s eta 0:00:02
  


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\dperez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [19]:
import mysql.connector
import sys
import xlsxwriter
import pandas as pd
import sys
import psycopg2
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME','root')
DB_DATABASE = os.getenv('DB_DATABASE','base_prueba')
DB_PASSWORD = os.getenv('DB_PASSWORD','')
DB_PORT = os.getenv('DB_PORT','3306')
DB_HOST=os.getenv('DB_HOST','127.0.0.1')

cnx = mysql.connector.connect(
    user=DB_USERNAME,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT,
    database=DB_DATABASE
    )
   

In [20]:
cur = cnx.cursor()

cur.execute("SELECT DATABASE();")
print(cur.fetchone())

('base_prueba',)


In [21]:
recados=pd.read_sql("SELECT * FROM recados",cnx)
egresados=pd.read_sql("SELECT * FROM egresados",cnx)
carreras=pd.read_sql("SELECT * FROM carreras",cnx)
usuarios=pd.read_sql("SELECT * FROM users",cnx)

C:\Users\dperez\AppData\Local\Temp\ipykernel_29048\548551767.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recados=pd.read_sql("SELECT * FROM recados",cnx)
C:\Users\dperez\AppData\Local\Temp\ipykernel_29048\548551767.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  egresados=pd.read_sql("SELECT * FROM egresados",cnx)
C:\Users\dperez\AppData\Local\Temp\ipykernel_29048\548551767.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  carreras=pd.read_sql("SELECT * FROM carreras",cnx)
C:\Users\dperez\AppData\Local\Temp\ip

In [22]:
muestras=pd.read_sql("SELECT * FROM muestras where estudio_id='3'",cnx)

C:\Users\dperez\AppData\Local\Temp\ipykernel_29048\3431015163.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  muestras=pd.read_sql("SELECT * FROM muestras where estudio_id='3'",cnx)


In [23]:
egresados=pd.read_sql("SELECT * FROM egresados",cnx)

C:\Users\dperez\AppData\Local\Temp\ipykernel_29048\591634635.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  egresados=pd.read_sql("SELECT * FROM egresados",cnx)


In [24]:
recados_merge=recados.merge(egresados[['cuenta','carrera','plantel']],how='inner',on='cuenta')

In [26]:
import datetime
#Establecer umbral de vuelta (cuanto se entiende que es el intervalo de descanso minimo)

tresh=19 #22 dias naturales¿?
hoy=datetime.datetime.now()

In [27]:
#filtrar recados
carrera='101'
plantel='3'
rec=recados_merge.loc[(recados_merge['carrera']==carrera)&(recados_merge['plantel']==plantel)]
print(len(rec))

0


In [28]:
655/2

327.5

In [29]:
import datetime

In [30]:
ordered_rec=rec.sort_values(by='fecha',ascending=True)
#dar formato a las fechas ordenadas
ordered_rec['fecha']=ordered_rec['fecha'].astype(str).apply(lambda x: x[:10])

In [31]:
for k in range(len(muestras)):
    #filtrar recados
    carrera=str(muestras['carrera_id'].values[k])
    plantel=str(muestras['plantel_id'].values[k])
    rec=recados_merge.loc[(recados_merge['carrera']==carrera)&(recados_merge['plantel']==plantel)]
    print('carrera',carrera,plantel,len(rec))
    fecha_inicial=datetime.datetime.strptime(str(ordered_rec['fecha'].values[0]),'%Y-%m-%d')
    nvuelta=1
    print(fecha_inicial)
    for i in range(len(ordered_rec)):
        this_fecha=datetime.datetime.strptime(str(ordered_rec['fecha'].values[i]),'%Y-%m-%d')
        distance=this_fecha-fecha_inicial
        if(distance.days>tresh):
            print('esta ya viene a ser otra vuelta',i,this_fecha)
            #Ingresar fehca inicio de la vuelta a la base
            cur.execute("INSERT INTO vueltas(muestra_id,inicio,numero_vuelta,created_at) VALUES (%s, %s,%s,%s)", (str(muestras['id'].values[k]),str(fecha_inicial),str(nvuelta),str(hoy))) 
            fecha_inicial=this_fecha
            nvuelta=nvuelta+1
        # print(distance,ordered_rec['fecha'].values[i])
    cur.execute("INSERT INTO vueltas(muestra_id,inicio,numero_vuelta,created_at) VALUES (%s, %s,%s,%s)", (str(muestras['id'].values[k]),fecha_inicial,str(nvuelta),hoy)) 
            

carrera 102 1 29


IndexError: index 0 is out of bounds for axis 0 with size 0

In [135]:
vueltas=pd.read_sql('select * from vueltas',cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [136]:
vueltas['numero_vuelta'].unique()

array(['1', '2', '3'], dtype=object)

In [137]:
cnx.commit()

In [52]:
datetime.datetime.strptime('Jun 1 2005', '%b %d %Y').date()

datetime.date(2005, 6, 1)

176

In [4]:
reactivos=pd.read_sql('select * from reactivos',cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [1]:
for i in range(len(reactivos)):
    print(reactivos['clave'].values[i],reactivos['description'].values[i],reactivos['type'].values[i],)

NameError: name 'reactivos' is not defined

In [2]:
reactivos.orden.unique()

NameError: name 'reactivos' is not defined